In [6]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

from time import sleep
from random import randint

headers = {"Accept-Language": "en-US,en;q=0.5"}

titles = []
years = []
time = []
imdb_ratings = []
metascores = []
votes = []
us_gross = []

pages = np.arange(1, 1001, 50)

for page in pages:
    page = requests.get("https://www.imdb.com/search/title/?groups=top_1000&start=" + str(page) + "&ref_=adv_nxt", headers=headers)

    soup = BeautifulSoup(page.text, 'html.parser')

    movie_div = soup.find_all('div', class_='lister-item mode-advanced')

    sleep(randint(2,10))
    
    for container in movie_div:
    
        name = container.h3.a.text

        titles.append(name)

        year = container.h3.find('span', class_='lister-item-year').text
        years.append(year)

        runtime = container.p.find('span', class_='runtime').text if container.p.find('span', class_='runtime') else '-'
        time.append(runtime)

        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)

        m_score = container.find('span', class_='metascore').text if container.find('span', class_='metascore') else '-'
        metascores.append(m_score)

        nv = container.find_all('span', attrs={'name': 'nv'})

        vote = nv[0].text
        votes.append(vote)

        grosses = nv[1].text if len(nv) > 1 else '-'
        us_gross.append(grosses)
        
        movies = pd.DataFrame({
            'movie': titles,
            'year': years,
            'timeMin': time,
            'imdb': imdb_ratings,
            'metascore': metascores,
            'votes': votes,
            'us_grossMillions': us_gross,
            })

        movies['year'] = movies['year'].str.extract('(\d+)').astype(int)

        movies['timeMin'] =  pd.to_numeric(movies['metascore'], errors='coerce')

        movies['votes'] = movies['votes'].str.replace(',', '').astype(int)

        movies['metascore'] = movies['metascore'].str.extract('(\d+)')
        movies['metascore'] = pd.to_numeric(movies['metascore'], errors='coerce')

        movies['us_grossMillions'] = movies['us_grossMillions'].map(lambda x: x.lstrip('$').rstrip('M'))
        movies['us_grossMillions'] = pd.to_numeric(movies['us_grossMillions'], errors='coerce')

      
movies.to_csv('movies.csv')

In [16]:

df = pd.read_csv("movies.csv")
df.drop(df.columns[0], axis=1,inplace =True)

In [17]:
df

,movie,year,timeMin,imdb,metascore,votes,us_grossMillions
0,The Mitchells vs the Machines,2021,80.0,7.8,80.0,52906,NaN
1,Zack Snyder's Justice League,2021,54.0,8.1,54.0,296269,NaN
2,The Father,2020,88.0,8.3,88.0,65007,NaN
3,Avengers: Endgame,2019,78.0,8.4,78.0,881584,858.37
4,The Shawshank Redemption,1994,80.0,9.3,80.0,2400880,28.34
...,...,...,...,...,...,...,...
995,The Circus,1928,90.0,8.1,90.0,30994,NaN
996,Vizontele,2001,NaN,8.0,NaN,34482,NaN
997,Black,2005,NaN,8.2,NaN,33755,0.73
998,The Breath,2009,NaN,8.0,NaN,32337,NaN
